In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [2]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D,GRU
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
import numpy as np
import pandas as pd

In [3]:
EMBEDDING_FILE = "/mnt/c/Users/lokes/Downloads/glove.840B.300d.txt"
train = pd.read_csv("/mnt/c/Users/lokes/Downloads/train.tsv",delimiter="\t").fillna(" ")

In [4]:
max_features = 100000
maxlen = 150
embed_size=300

In [5]:
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate


In [7]:
train["Label"].value_counts()

moderate          6019
not depression    1971
severe             901
Name: Label, dtype: int64

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["Label"] = le.fit_transform(train["Label"])

In [9]:
train["Label"].value_counts()

0    6019
1    1971
2     901
Name: Label, dtype: int64

In [10]:
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,0
1,train_pid_2,My new years resolution : I'm gonna get my ass...,0
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,0
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",0
4,train_pid_5,Sat in the dark and cried myself going into th...,0


In [11]:
df=train[0:901]
df2=train[6019:6920]
df3=train[7990:]

df = df[['Text_data', 'Label']]
df2 = df2[['Text_data', 'Label']]
df3 = df3[['Text_data', 'Label']]

print("Moderate cases: ", str(len(df)))
print("Not Depressed cases: ", str(len(df2)))
print("Severe cases: ", str(len(df3)))

data = pd.concat([df, df2, df3], axis=0).reset_index(drop=True)

Moderate cases:  901
Not Depressed cases:  901
Severe cases:  901


In [12]:
train_x = data["Text_data"].str.lower()
train_y = data["Label"]
train_y = pd.get_dummies(train_y,columns=train_y).values
train_y.shape

(2703, 3)

In [13]:
test_x =test["text data"].str.lower()

In [14]:
tokenizer = text.Tokenizer(num_words=max_features,lower=True)
tokenizer.fit_on_texts(list(train_x))

In [15]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [16]:
train_x = sequence.pad_sequences(train_x,maxlen=maxlen)
test_x = sequence.pad_sequences(test_x,maxlen=maxlen)

In [17]:
embedding_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.strip().rsplit(" ")
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embedding_index[word] = coefs

In [18]:
word_index = tokenizer.word_index
num_words = min(len(word_index)+1,max_features)
embedding_matrix = np.zeros((num_words,embed_size))

for word, i in word_index.items():
    if i>=max_features:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Basic N network

In [46]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.35)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 8s 62ms/step - loss: 0.8552 - accuracy: 0.5923
Epoch 2/5
85/85 [==============================] - 5s 61ms/step - loss: 0.4861 - accuracy: 0.8239
Epoch 3/5
85/85 [==============================] - 5s 60ms/step - loss: 0.2897 - accuracy: 0.9031
Epoch 4/5
85/85 [==============================] - 5s 59ms/step - loss: 0.1763 - accuracy: 0.9475
Epoch 5/5
85/85 [==============================] - 5s 62ms/step - loss: 0.1195 - accuracy: 0.9626


In [22]:
new_complaint =['not feeling happy']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = sequence.pad_sequences(seq, maxlen=maxlen, truncating='post', padding='post')
pred = model.predict(padded)
labels = ['moderate','not depression','severe']
print(pred, labels[np.argmax(pred)])

[[0.93639684 0.05198456 0.01161859]] moderate
